In [52]:
#6
#athena db config - lifearchi
#AAI-540 Group 3 FP

### Notebook 6

This notebook registers the LifeArchitect model specification dataset in Amazon Athena for SQL-based analysis.  
The dataset is stored in Amazon S3 and exposed as an external table without duplicating data.  
This enables querying model attributes such as parameter size, benchmark metrics, and architecture tags.  
The resulting table supports downstream model profiling and routing analysis.

In [53]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd

### Configure AWS and Athena Environment
Initialize AWS session, identify the project bucket, and configure the Athena staging directory for query outputs.

In [54]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
ingest_create_athena_db_passed = False

In [55]:
database_name = "dsoaws"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

### Create or Select Athena Database
Ensure the Athena database exists so external tables can be registered consistently.

In [56]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS dsoaws


/tmp/ipykernel_2604/2532319348.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [57]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_2604/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws
2,sagemaker_featurestore


### Define S3 Data Location for Athena Table
Set the dataset source path and the dedicated S3 folder that Athena will use as the table location.

In [58]:
s3_data_path = f"s3://{bucket}/lifearchitectmodels.csv"
s3_table_path = f"s3://{bucket}/table3/"
print("s3_data_path:", s3_data_path)
print("s3_table_path:", s3_table_path)

s3_data_path: s3://sagemaker-us-east-1-907086662522/lifearchitectmodels.csv
s3_table_path: s3://sagemaker-us-east-1-907086662522/table3/


### Prepare Table Data Directory in S3
Copy the dataset into the S3 folder referenced by the Athena external table.

In [59]:
# Download original file from S3 (source)
!aws s3 cp {s3_data_path} /tmp/lifearchitectmodels_src.csv

import pandas as pd

df = pd.read_csv("/tmp/lifearchitectmodels_src.csv")

# Remove the embedded header row that shows up as a normal record
# (the bad row has model == "Model")
df = df[df["model"].astype(str).str.strip() != "Model"]

# Save cleaned file
clean_path = "/tmp/lifearchitectmodels_clean.csv"
df.to_csv(clean_path, index=False)

# Upload cleaned file into the Athena table folder (overwrite the one in table3/)
!aws s3 cp {clean_path} {s3_table_path}lifearchitectmodels.csv

print("Cleaned lifearchitectmodels.csv uploaded to table3/")

download: s3://sagemaker-us-east-1-907086662522/lifearchitectmodels.csv to ../../../../tmp/lifearchitectmodels_src.csv
upload: ../../../../tmp/lifearchitectmodels_clean.csv to s3://sagemaker-us-east-1-907086662522/table3/lifearchitectmodels.csv
Cleaned lifearchitectmodels.csv uploaded to table3/


### Verify Dataset Upload
Confirm the dataset exists inside the table directory before creating the external table.

In [60]:
#table3 - lifearch
table_name_csv = 'lifearchitect'
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
ingest_create_athena_table_csv_passed = False
!aws s3 cp {s3_table_path}lifearchitectmodels.csv /tmp/life_table3.csv
dataexplore = pd.read_csv("/tmp/life_table3.csv")
dataexplore.head()

download: s3://sagemaker-us-east-1-907086662522/table3/lifearchitectmodels.csv to ../../../../tmp/life_table3.csv


,model,lab,parameters_b,tokens_trained_b,ratio_tokens_params,alscore,mmlu,mmlu_pro,gpqa,hle,announced,public,arch,tags
0,AuroraGPT (ScienceGPT),Argonne National Laboratory,2000.0,30000.0,15:1,25.8,NaN,NaN,NaN,NaN,TBA,🔴,NaN,NaN
1,Avocado,Meta AI,2000.0,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,TBA,NaN,NaN,NaN
2,DeepSeek-R2,DeepSeek-AI,1200.0,15640.0,14:1,14.4,NaN,NaN,NaN,NaN,TBA,🟢,MoE,"Reasoning, SOTA"
3,GPT-6,OpenAI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TBA,NaN,NaN,SOTA
4,Grok-5,xAI,6000.0,100000.0,17:1,81.6,NaN,NaN,NaN,NaN,TBA,NaN,MoE,NaN


In [61]:
dataexplore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   model                753 non-null    object 
 1   lab                  752 non-null    object 
 2   parameters_b         734 non-null    float64
 3   tokens_trained_b     660 non-null    float64
 4   ratio_tokens_params  657 non-null    object 
 5   alscore              657 non-null    float64
 6   mmlu                 248 non-null    float64
 7   mmlu_pro             144 non-null    float64
 8   gpqa                 191 non-null    float64
 9   hle                  52 non-null     float64
 10  announced            752 non-null    object 
 11  public               749 non-null    object 
 12  arch                 748 non-null    object 
 13  tags                 189 non-null    object 
dtypes: float64(7), object(7)
memory usage: 82.5+ KB


In [62]:
!aws s3 ls {s3_table_path}

2026-02-22 23:45:59      51640 lifearchitectmodels.csv


### Create Athena External Table
Define the schema and register the dataset with Athena using an external table pointing to the S3 table directory.

In [63]:
# Drop and recreate the external table so Athena reads the cleaned file
pd.read_sql(f"DROP TABLE IF EXISTS {database_name}.{table_name_csv}", conn)

statement = f"""CREATE EXTERNAL TABLE {database_name}.{table_name_csv}(
  model STRING,
  lab STRING,
  parameters_b STRING,
  tokens_trained_b STRING,
  ratio_tokens_params STRING,
  alscore STRING,
  mmlu STRING,
  mmlu_pro STRING,
  gpqa STRING,
  hle STRING,
  announced STRING,
  public STRING,
  arch STRING,
  tags STRING
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  'separatorChar' = ',',
  'quoteChar'     = '\"',
  'escapeChar'    = '\\\\'
)
LOCATION '{s3_table_path}'
TBLPROPERTIES ('skip.header.line.count'='1')
"""

/tmp/ipykernel_2604/3945074618.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(f"DROP TABLE IF EXISTS {database_name}.{table_name_csv}", conn)


In [64]:
pd.read_sql(statement, conn)

/tmp/ipykernel_2604/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [65]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_2604/84333453.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,aimodelpoll
1,amazon_reviews_parquet
2,amazon_reviews_tsv
3,lifearchitect
4,llmachievements


### Validate Table Access
Run a sample query (LIMIT 10) to verify that Athena can successfully read the dataset.


In [66]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)
df = pd.read_sql(statement, conn)
df.head()

SELECT * FROM dsoaws.lifearchitect LIMIT 10


/tmp/ipykernel_2604/888292236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,model,lab,parameters_b,tokens_trained_b,ratio_tokens_params,alscore,mmlu,mmlu_pro,gpqa,hle,announced,public,arch,tags
0,AuroraGPT (ScienceGPT),Argonne National Laboratory,2000,30000,15:1,25.8,,,,,TBA,🔴,,
1,Avocado,Meta AI,2000,30000,,,,,,,TBA,,,
2,DeepSeek-R2,DeepSeek-AI,1200,15640,14:1,14.4,,,,,TBA,🟢,MoE,"Reasoning, SOTA"
3,GPT-6,OpenAI,,,,,,,,,TBA,,,SOTA
4,Grok-5,xAI,6000,100000,17:1,81.6,,,,,TBA,,MoE,


### Summary

This notebook created an Athena external table over the LifeArchitect model specification dataset stored in Amazon S3.

The table schema is registered in the AWS Glue Data Catalog while the underlying data remains in S3, enabling serverless querying without data movement.

The resulting table provides structured access to model capability attributes that support model comparison, profiling, and cost-aware routing analysis.
